# **Example 6** Validation of ATLID level 2 data

This notebook shows how A-EBD profiles (backscatter, extinction, lidar ratio and depol. ratio) can be compared with ground-based data from `.nc`-files.

In [ ]:
import earthcarekit as eck

## Requirements

For this example you need a A-EBD file from an overpass and the matching ground data (e.g., PollyNET `.nc`-file).

Paste both file paths into the cell below:

In [ ]:
fp_aebd = r"./ECA_EXBA_ATL_EBD_2A_20240902T210023Z_20250721T110708Z_01508B.h5"
fp_ground = r"./2024_09_02_Mon_TJK_00_00_01_2000_2130_profiles.nc"

# Also specify the ground site by either setting its name (string) or creating a custom GroundSite object:
site = "dushanbe"
radius_km = 100.0

## **6.1** Open and view datasets

In [ ]:
print("Open the A-EBD dataset:")
with eck.read_product(fp_aebd) as ds_aebd:
    display(ds_aebd)

print("Open the ground-based dataset:")
with eck.read_nc(fp_ground) as ds_ground:
    display(ds_ground)

In [ ]:
import logging
print("Create a quicklook of the overpass from the A-EBD file:")
print("Note: When plotting profiles from A-PRO products, the `ecquicklook` function will automatically plot only the closest profile within the set radius.")
fig, _ = eck.ecquicklook(
    ds=ds_aebd,
    site=site,
    radius_km=radius_km,
    resolution="low",  # Change the A-EBD resolution as needed
    height_range=(0, 12e3),  # Change the plotting height as needed
    selection_max_time_margin="00:01:00",  # Crop the plot to show only 1 Minute around the overpass start and end times
    logger=logging.getLogger(),  # Optional: Shows plotting progress
)
print("Save the quicklook:")
eck.save_plot(fig, filepath="./01508B_20250902_aebd_quicklook.png")

## **Example 6.2** Create backscatter, extinction, lidar ratio and depol. ratio profile plots and comparison statistics 

In [ ]:
fig, df = eck.compare_bsc_ext_lr_depol(
    fp_ec=fp_aebd,
    fp_target=fp_ground,
    site=site,
    time_var_target="start_time",  # Set to the time variable name in the ground-based dataset
    height_var_target="height",  # Set to the height variable name in the ground-based dataset
    resolustion="low",  # Change the A-EBD resolution as needed
    bsc_var_target=["aerBsc_klett_355", "aerBsc_raman_355"],  # Give at least one bsc. variable name from the ground-based dataset
    ext_var_target=["aerExt_klett_355", "aerExt_raman_355"],  # Give at least one ext. variable name from the ground-based dataset
    lr_var_target=["aerLR_klett_355", "aerLR_raman_355"],  # Give at least one LR variable name from the ground-based dataset
    depol_var_target=["parDepol_klett_355", "parDepol_raman_355"],  # Give at least one depol. variable name from the ground-based dataset
    height_range=(0, 12e3),  # Change the plotting height as needed
    selection_height_range=(2e3, 4e3),  # Change the selected height range to get statistics from as needed or set to None
)
display(df)

print("Save the plot:")
eck.save_plot(fig, filepath="./01508B_20250902_aebd_vs_polly_profiles.png")

print("Save the stat. results:")
df.to_csv("./01508B_20250902_aebd_vs_polly_profiles.csv", index=False)